In [ ]:
#%rm -r dataset/

In [ ]:
import os
from PIL import Image, ImageFilter
import numpy as np
import matplotlib.pyplot as plt
import glob
import random
from keras.preprocessing.image import array_to_img
import cv2

Using TensorFlow backend.


In [ ]:
verbose = True
class Log:
    verbose = True
    @classmethod
    def info(cls, msg):
        if cls.verbose:
            print(msg)

In [ ]:
def download_ds(download_address, save_address):
    if os.path.exists(save_address):
        Log.info("The dataset has already been downloaded.")
    else:
        !wget -O ds.zip {download_address}
        !unzip ds.zip
        !mv ShapesData_few-show-learning {save_address}
        !find {save_address} -name '*.DS_Store' -delete

In [ ]:
download_address = "https://www.dropbox.com/s/9imgl9m0qzin55g/ShapesData_few-show-learning.zip?dl=0"
save_address = "dataset"
download_ds(download_address, save_address)

Streaming output truncated to the last 5000 lines.
  inflating: __MACOSX/ShapesData_few-show-learning/batch1/House/._House_Yellow_Artificial_0_frame35.jpg  
  inflating: ShapesData_few-show-learning/batch1/House/House_Green_Vertical_Morning_0_frame755.jpg  
  inflating: __MACOSX/ShapesData_few-show-learning/batch1/House/._House_Green_Vertical_Morning_0_frame755.jpg  
  inflating: ShapesData_few-show-learning/batch1/House/House_Yellow_Artificial_0_frame34.jpg  
  inflating: __MACOSX/ShapesData_few-show-learning/batch1/House/._House_Yellow_Artificial_0_frame34.jpg  
  inflating: ShapesData_few-show-learning/batch1/House/House_Orange_Vertical_Morning_0_frame1415.jpg  
  inflating: __MACOSX/ShapesData_few-show-learning/batch1/House/._House_Orange_Vertical_Morning_0_frame1415.jpg  
  inflating: ShapesData_few-show-learning/batch1/House/House_Yellow_Vertical_Morning_0_frame338.jpg  
  inflating: __MACOSX/ShapesData_few-show-learning/batch1/House/._House_Yellow_Vertical_Morning_0_frame338.jpg

In [ ]:
def load_ds(address, new_im_size, channels):
    x, y = [], []
    label_idx = dict()
    if os.path.exists(os.path.join(address,".DS_Store")):
        os.remove(os.path.join(address, ".DS_Store"))
    for i , label in enumerate(os.listdir(address)):
        label_idx[i] = label
        label_address = os.path.join(address, label)
        
        for image_name in os.listdir(label_address):
            image_address = os.path.join(label_address, image_name)
            #try:
            img = Image.open(image_address).convert(channels)
            img = np.array(img)                            
            #print(img.shape)

            if (len(img.shape))==2:
                image2= np.zeros((img.shape[0], img.shape[1], 1))
                image2[:,:,0]=img[:,:]
                img=image2
                
            img = np.resize(img, (new_im_size, new_im_size,img.shape[2]))
                

            

            x.append(img)               
            y.append(i)
            #except:
               # Log.info("ERROR: can't load {}".format(image_address))
            
    #Log.info(len(x))
    #Log.info(len(y))
    return x,y

In [ ]:
new_im_size=105
batch1_address = os.path.join(save_address, "batch1")
batch2_address = os.path.join(save_address,"batch2_gray")
train_X1, train_Y1 = load_ds(batch1_address,new_im_size,"RGB")
train_X2, train_Y2 = load_ds(batch2_address,new_im_size,"L")

print("train_X1.size = ",len(train_X1))
print("train_X2.size = ",len(train_X2))
 

train_X1.size =  7090
train_X2.size =  256


In [ ]:
#Normalization
train_X1 = np.asarray(train_X1, dtype=np.float32) / 255
train_X2 = np.asarray(train_X2, dtype=np.float32) / 255                                        

In [ ]:
# Standardizing the data
def standardize_dataset(X):
    image_means = []
    image_stds = []

    for image in X:
        image_means.append(np.mean(image))                                            # Computing the image mean
        image_stds.append(np.std(image))                                            # Computing the image standard deviation

    dataset_mean = np.mean(image_means)                                                # Computing the dataset mean
    dataset_std = np.mean(image_stds)                                                # Computing the dataset standard deviation
    return [dataset_mean, dataset_std]                    # For every image we subtract to it the dataset mean and we divide by the dataset standard deviation

In [ ]:
dataset_mean1, dataset_std1 = standardize_dataset(train_X1)
train_X1 = (train_X1-dataset_mean1)/dataset_std1

dataset_mean2, dataset_std2 = standardize_dataset(train_X2)
train_X2 = (train_X2-dataset_mean2)/dataset_std2

In [ ]:
print("train_X1.shape = ",train_X1.shape)
print("train_X2.shape = ",train_X2.shape)

train_X1.shape =  (7090, 105, 105, 3)
train_X2.shape =  (256, 105, 105, 1)


In [ ]:
# Data augmentation
# The augm_factor determine the copy amount created from every single image

def augment_dataset(data_dir_path, augm_factor, categories):
    for i in range(len(categories)):
        category = categories[i]

        only_jpg = glob.glob(data_dir_path + "/" + category + "/*.jpg")                # Getting all the .jpg images
        only_png = glob.glob(data_dir_path + "/" + category + "/*.png")                # Getting all the .png images
        only_images = only_jpg + only_png                                            # Merging all the images 
        #print(len(only_images))

      

        for file_name in only_images:                                                # For every image in the dataset
            image = Image.open(file_name)
            #image = cv2.imread(file_name)
            image = image.resize((new_im_size, new_im_size))                        # Resize the image
            image.save(file_name)                                                    # Save the image with new size
            rows,cols=image.size

            #print("test")

            for j in range(augm_factor):                                            # We create a new image checking the augmentation factor

                translation_x=np.random.randint(40,60)
                translation_y=np.random.randint(40,60)
                scale=np.random.randint(150,200)
                rotation_2d=np.random.uniform(0,70)
                rotation_3dx=np.random.uniform(50,100)
                rotation_3dy=np.random.uniform(130,200)

                pts1 = np.float32([[50,50],[200,50],[50,200]])
                pts2 = np.float32([[translation_x,translation_y],[scale,rotation_2d],[rotation_3dx,rotation_3dy]])
                #print(pts2)
                M = cv2.getAffineTransform(pts1,pts2)
                #print(M)
                #plt.imshow(image)
                image = np.array(image)
                dst = cv2.warpAffine(image,M,(cols,rows))

                #print(dst.shape)
                if (len(dst.shape))==2:
                    image2= np.zeros((dst.shape[0], dst.shape[1], 1))
                    image2[:,:,0]=dst[:,:]
                    dst=image2
                    #print(dst.shape)

                dst=array_to_img(dst)


            


                dst.save(file_name[:-4] + "_augm_" + str(j) + file_name[-4:])        # Saving the new, modified, image

In [ ]:
# Augmentation parameters
address="dataset/batch1/Bat"
train_dir1="dataset/batch1"
train_dir2="dataset/batch2_gray"
categories1 = ["Bat", "Boy", "Car", "Crocodile", "Girl", "House", "If", "Lion", "Monkey", "Tree"]
categories2 = ["Bat", "Boy", "Car", "Crocodile", "Generic", "Girl", "House", "If", "Lion", "Monkey", "Tree"]
augment_dataset(train_dir1,1, categories1)            # Augmenting the training set
augment_dataset(train_dir2,1, categories2)

In [ ]:
new_im_size=105
batch1_address = os.path.join(save_address, "batch1")
train_X1, train_Y1 = load_ds(batch1_address,new_im_size,"RGB")
print(len(train_X1))
batch2_address = os.path.join(save_address, "batch2_gray")
train_X2, train_Y2 = load_ds(batch2_address,new_im_size,"L")
print(len(train_X2))

14180
512


In [ ]:
#Normalization
train_X1 = np.asarray(train_X1, dtype=np.float32) / 255 
train_X2 = np.asarray(train_X2, dtype=np.float32) / 255    

In [ ]:
# Standardizing the data
def standardize_dataset(X):
    image_means = []
    image_stds = []

    for image in X:
        image_means.append(np.mean(image))                                            # Computing the image mean
        image_stds.append(np.std(image))                                            # Computing the image standard deviation

    dataset_mean = np.mean(image_means)                                                # Computing the dataset mean
    dataset_std = np.mean(image_stds)                                                # Computing the dataset standard deviation
    return [dataset_mean, dataset_std]                    # For every image we subtract to it the dataset mean and we divide by the dataset standard deviation

In [ ]:
dataset_mean1, dataset_std1 = standardize_dataset(train_X1)
train_X1 = (train_X1-dataset_mean1)/dataset_std1
dataset_mean2, dataset_std2 = standardize_dataset(train_X2)
train_X2 = (train_X2-dataset_mean2)/dataset_std2

In [ ]:
print(train_X1.shape)
print(train_X2.shape)

(14180, 105, 105, 3)
(512, 105, 105, 1)


In [ ]:
from google.colab import files
!zip -r Ads.zip dataset 
files.download('Ads.zip') 

Streaming output truncated to the last 5000 lines.
  adding: dataset/batch1/Girl/Girl_Green_Vertical_Morning_0_frame514.jpg (deflated 11%)
  adding: dataset/batch1/Girl/Girl_Green_Vertical_Morning_0_frame342_augm_0.jpg (deflated 10%)
  adding: dataset/batch1/Girl/Girl_Green_Vertical_Morning_0_frame135_augm_0.jpg (deflated 7%)
  adding: dataset/batch1/Girl/Girl_Green_Vertical_Morning_0_frame1921_augm_0.jpg (deflated 11%)
  adding: dataset/batch1/Girl/Girl_LightBlue_Vertical_Morning_0_frame357.jpg (deflated 11%)
  adding: dataset/batch1/Girl/Girl_Yellow_Morning_0_frame668_augm_0.jpg (deflated 9%)
  adding: dataset/batch1/Girl/Girl_Red_Artificial_0_frame676.jpg (deflated 14%)
  adding: dataset/batch1/Girl/Girl_Green_Vertical_Morning_0_frame1392_augm_0.jpg (deflated 8%)
  adding: dataset/batch1/Girl/Girl_Green_Vertical_Morning_0_frame520.jpg (deflated 12%)
  adding: dataset/batch1/Girl/Girl_LightBlue_Artificial_0_frame1237.jpg (deflated 12%)
  adding: dataset/batch1/Girl/Girl_Green_Vertica

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>